## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb　←いまここ

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

## 課題と機関と研究代表者をつなぐ

In [26]:
# encoding: utf-8
import pandas as pd
import numpy as np
import pickle

### 課題を読み込む

In [27]:
with open('afterCleaning/parse_kadai.dump', 'rb') as f:
    kadai = pickle.load(f)

In [28]:
kadai

,projecttype,category,category_niicode,section,section_niicode,startfiscalyear,endfiscalyear,directcost,title_ja,title_en
awardnumber,,,,,,,,,,
S003,area,新学術領域研究(研究領域提案型),73,0,0,2010,2015,0,包括型脳科学研究推進支援ネットワーク,0
S002,area,新学術領域研究(研究領域提案型),73,0,0,2010,2015,0,ゲノム科学の総合的推進に向けた大規模ゲノム情報生産・高度情報解析支援,0
S001,area,新学術領域研究(研究領域提案型),73,0,0,2010,2015,0,がん研究分野の特性等を踏まえた支援活動,0
4203,area,新学術領域研究(研究領域提案型),73,0,0,2010,2014,0,統合的多階層生体機能学領域の確立とその応用,Establishment of Integrative Multi-level Syste...
4202,area,新学術領域研究(研究領域提案型),73,0,0,2010,2014,0,質感認知の脳神経メカニズムと高度質感情報処理技術の融合的研究,Integrative studies of neural mechanisms and a...
4201,area,新学術領域研究(研究領域提案型),73,0,0,2010,2014,0,システム的統合理解に基づくがんの先端的診断、治療、予防法の開発,Integrative Systems Understanding of Cancer fo...
3224,area,新学術領域研究(研究領域提案型),73,0,0,2010,2014,0,先端技術を駆使したＨＬＡ多型・進化・疾病に関する統合的研究,"HLA polymorphism, disease and evolution"
3223,area,新学術領域研究(研究領域提案型),73,0,0,2010,2014,0,性差構築の分子基盤,Molecular mechanisms for establishment of sex ...
3222,area,新学術領域研究(研究領域提案型),73,0,0,2010,2014,0,ゲノム複製・修復・転写のカップリングと普遍的なクロマチン構造変換機構,"Coupling of replication, repair and transcript..."


In [29]:
kadai = kadai[['category_niicode', 'section_niicode','startfiscalyear', 'directcost']]
kadai

,category_niicode,section_niicode,startfiscalyear,directcost
awardnumber,,,,
S003,73,0,2010,0
S002,73,0,2010,0
S001,73,0,2010,0
4203,73,0,2010,0
4202,73,0,2010,0
4201,73,0,2010,0
3224,73,0,2010,0
3223,73,0,2010,0
3222,73,0,2010,0


### 研究機関を読み込む

In [30]:
with open('afterCleaning/parse_institution_from_grantlist.dump', 'rb') as f:
    institution = pickle.load(f)

In [31]:
institution

,fiscalyear,grant_sequence,institution_sequence,institution_niicode,institution_mextcode,institution_jspscode,institution
awardnumber,,,,,,,
08F08045,2008,1,1,17401,17401,17401,熊本大学
08J03857,2008,1,1,14401,14401,14401,大阪大学
09J02906,2009,1,1,14301,14301,14301,京都大学
09J06155,2009,1,1,11301,11301,11301,東北大学
09J06174,2009,1,1,12601,12601,12601,東京大学
09J08442,2009,1,1,13901,13901,13901,名古屋大学
09J10929,2009,1,1,13201,13201,13201,富山大学
09J55242,2009,1,1,13601,13601,13601,信州大学
10F00002,2010,1,1,11301,11301,11301,東北大学


In [32]:
institution = institution[['institution_niicode']]
institution

,institution_niicode
awardnumber,
08F08045,17401
08J03857,14401
09J02906,14301
09J06155,11301
09J06174,12601
09J08442,13901
09J10929,13201
09J55242,13601
10F00002,11301


### 研究代表者を読み込む

In [33]:
with open('afterCleaning/parse_member_from_summary.dump', 'rb') as f:
    member = pickle.load(f)

In [34]:
member

,sequence,role,kenkyuusha_id,familyname,givenname
awardnumber,,,,,
1201,1,area_organizer,70013753,赤澤,威
1301,1,area_organizer,172255,仲,真紀子
1401,1,area_organizer,70192648,常木,晃
1501,1,area_organizer,70254133,園部,哲史
15H00001,1,principal_investigator,0,江谷,和樹
15H00002,1,principal_investigator,0,北田,聖子
15H00003,1,principal_investigator,0,多田,英俊
15H00004,1,principal_investigator,0,國本,学史
15H00005,1,principal_investigator,0,北川,美穂


In [35]:
member = member[['kenkyuusha_id']]
member

,kenkyuusha_id
awardnumber,
1201,70013753
1301,172255
1401,70192648
1501,70254133
15H00001,0
15H00002,0
15H00003,0
15H00004,0
15H00005,0


### 課題と機関と研究代表者をJOINする

In [36]:
df = kadai.join(institution)
df = df.join(member)
df

,category_niicode,section_niicode,startfiscalyear,directcost,institution_niicode,kenkyuusha_id
awardnumber,,,,,,
S003,73,0,2010,0,0,40118451.0
S002,73,0,2010,0,0,70135292.0
S001,73,0,2010,0,0,60117603.0
4203,73,0,2010,0,0,30142011.0
4202,73,0,2010,0,0,153669.0
4201,73,0,2010,0,0,50128104.0
3224,73,0,2010,0,0,50014121.0
3223,73,0,2010,0,0,30183114.0
3222,73,0,2010,0,0,50012670.0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258813 entries, S003 to 18K08432
Data columns (total 6 columns):
category_niicode       258813 non-null int64
section_niicode        258813 non-null int64
startfiscalyear        258813 non-null int64
directcost             258813 non-null int64
institution_niicode    258813 non-null int64
kenkyuusha_id          234177 non-null float64
dtypes: float64(1), int64(5)
memory usage: 23.8+ MB


In [38]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258813 entries, S003 to 18K08432
Data columns (total 6 columns):
category_niicode       258813 non-null int64
section_niicode        258813 non-null int64
startfiscalyear        258813 non-null int64
directcost             258813 non-null int64
institution_niicode    258813 non-null int64
kenkyuusha_id          258813 non-null float64
dtypes: float64(1), int64(5)
memory usage: 23.8+ MB


In [39]:
df = df.astype({
    'kenkyuusha_id': np.int64
})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258813 entries, S003 to 18K08432
Data columns (total 6 columns):
category_niicode       258813 non-null int64
section_niicode        258813 non-null int64
startfiscalyear        258813 non-null int64
directcost             258813 non-null int64
institution_niicode    258813 non-null int64
kenkyuusha_id          258813 non-null int64
dtypes: int64(6)
memory usage: 13.8+ MB


In [40]:
df

,category_niicode,section_niicode,startfiscalyear,directcost,institution_niicode,kenkyuusha_id
awardnumber,,,,,,
S003,73,0,2010,0,0,40118451
S002,73,0,2010,0,0,70135292
S001,73,0,2010,0,0,60117603
4203,73,0,2010,0,0,30142011
4202,73,0,2010,0,0,153669
4201,73,0,2010,0,0,50128104
3224,73,0,2010,0,0,50014121
3223,73,0,2010,0,0,30183114
3222,73,0,2010,0,0,50012670


ローカルのMariaDBに保存する

In [42]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

In [43]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('grantaward', engine, if_exists='replace',
          dtype={
              'awardnumber': String(256),
              'category_niicode': Integer,
              'section': String(256),
              'startfiscalyear': Integer,
              'directcost': Integer,
              'niicode': Integer,
              'kenkyuusha_id': Integer,                 
                })

2018-05-10 14:03:16,584 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-05-10 14:03:16,585 INFO sqlalchemy.engine.base.Engine {}
2018-05-10 14:03:16,586 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-05-10 14:03:16,587 INFO sqlalchemy.engine.base.Engine {}
2018-05-10 14:03:16,587 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-05-10 14:03:16,588 INFO sqlalchemy.engine.base.Engine {}
2018-05-10 14:03:16,589 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-05-10 14:03:16,589 INFO sqlalchemy.engine.base.Engine {}
2018-05-10 14:03:16,590 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-05-10 14:03:16,590 INFO sqlalchemy.engine.base.Engine {}
2018-05-10 14:03:16,592 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-05-10 14

全データの保存に5秒くらいかかる。無事にコミットできたら終了。あとは分析に使う。